In [1]:
import torch
import pandas as pd 
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import os

In [2]:
os.chdir("D:\\meridianthe4\\PML\\Cases\\Wisconsin")

In [3]:
wisconsin = pd.read_csv("BreastCancer.csv")
le = LabelEncoder()
wisconsin['Class'] = le.fit_transform(wisconsin['Class'])
X, y = wisconsin.drop('Class', axis=1), wisconsin['Class'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25, stratify=y)
scl_x = MinMaxScaler()
X_train_scaled = scl_x.fit_transform(X_train)
X_test_scaled = scl_x.transform(X_test)
X_torch = torch.from_numpy(X_train_scaled)
y_torch = torch.from_numpy(y_train)

In [7]:
joint_dataset = TensorDataset(X_torch.float(), y_torch.long())
torch.manual_seed(25)
data_loader = DataLoader(joint_dataset, batch_size=100, shuffle=True, drop_last=False)
criterion = torch.nn.BCELoss()
model = nn.Sequential(nn.Linear(in_features=X_train_scaled.shape[1], out_features=5),
                      nn.ReLU(),
                      nn.Linear(5, 3),
                      nn.ReLU(),
                      nn.Linear(3, 1),
                      nn.Sigmoid())
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [14]:
for epoch in np.arange(0,100):
    for i, batch in enumerate(data_loader, 1):
        # Forward pass: Compute predicted y by passing a to the model
        y_pred_prob = model(batch[0].float())
        
        # Compute and print loss
        loss = criterion(y_pred_prob, batch[1].float().unsqueeze(1))

        # Zero gradients, perfore a backward puss, and update the weights.
        optimizer.zero_grad()

        # perform a backward pass (backpropagation)
        loss.backward()

        # Update the parameters
        optimizer.step()
        
    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 0.07299929112195969
Epoch 11, Loss: 0.12125027924776077
Epoch 21, Loss: 0.1032010167837143
Epoch 31, Loss: 0.06522469967603683
Epoch 41, Loss: 0.03444380685687065
Epoch 51, Loss: 0.07320503890514374
Epoch 61, Loss: 0.07885845750570297
Epoch 71, Loss: 0.09480956196784973
Epoch 81, Loss: 0.10042992234230042
Epoch 91, Loss: 0.028913266956806183


In [15]:
X_torch_test = torch.from_numpy(X_test_scaled)
y_pred_prob = model(X_torch_test.float())
y_pred_prob = y_pred_prob.detach().numpy()
y_pred = np.where(y_pred_prob >= 0.5, 1, 0)
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Log Loss: ", log_loss(y_test, y_pred_prob))

Accuracy:  0.9666666666666667
Log Loss:  0.09650790242105708
